In [ ]:
## In this example, we will study output data frame from pandora.py configuration
#### 1. Opening each data frame and check structure
#### 2. Collect POT and scale factor to the target POT
#### 3. Merge evtdf and mcnudf for further study
#### 4. Draw some plots for each slice and for each pfp

import os
import sys

import numpy as np
import math
import uproot as uproot
import pickle
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import ticker
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
from matplotlib import gridspec

# Add the head direcoty to sys.path
workspace_root = os.getcwd()  
sys.path.insert(0, workspace_root + "/../../")

# import this repo's classes
import pyanalib.pandas_helpers as ph


In [ ]:
## 1. Open each df
venv_path = os.getenv("PATH")
evtdf = pd.read_hdf('../../test_pandoradf.df', key='evt')
hdrdf = pd.read_hdf('../../test_pandoradf.df', key='hdr')
mcnudf = pd.read_hdf('../../test_pandoradf.df', key='mcnu')

In [ ]:
#### 1.1 Check evtdf structure
evtdf

In [ ]:
evtdf.slc.columns

In [ ]:
evtdf.pfp.columns

In [ ]:
#### 1.2 Check hdrdf structure
hdrdf

In [ ]:
#### 1.3 Check mcnudf structure
mcnudf

In [ ]:
mcnudf.columns

In [ ]:
## 2. Collect POT and scale factor to the target POT
this_pot = sum(hdrdf.pot)
print(this_pot)
target_POT = 3.0e18
POT_scale = target_POT / this_pot
print(POT_scale)

In [ ]:
evtdf.reset_index()

In [ ]:
evtdf.slc.tmatch.idx.value_counts()

In [ ]:
mcnudf.reset_index()

In [ ]:
mcnudf.columns

In [ ]:
## 3. Merge evtdf and mcnudf
#### 1) Merging is based on matching between slc.tmatch.idx of evtdf and rec.mc.nu..index of mcnudf.
####    For each entry (readout window), there could be multiple truth neutrino interactions and reconstructed slices
####    We want to match each truth neutrino interaction to a corresponding slice

matchdf = ph.multicol_merge(evtdf.reset_index(), mcnudf.reset_index(),
                            left_on=[("entry", "",""), ("slc","tmatch", "idx", "", "", "")],
                            right_on=[("entry", "",""), ("rec.mc.nu..index", "","")], 
                            how="left") ## -- save all sllices

In [ ]:
matchdf

In [ ]:
matchdf.columns

In [ ]:
## 4. Draw plots
#### 4.1) Make dataframe of nu.E for each rec.slc..index (nuE_per_slc) and for each rec.slc.reco.pfp..index (nuE_per_pfp)
nuE_col = ('E', '', '', '', '', '')
nuE_per_slc = matchdf.groupby([('entry'), ('rec.slc..index')])[[nuE_col]].first()
nuE_per_pfp = matchdf.groupby([('entry'), ('rec.slc..index'), ('rec.slc.reco.pfp..index')])[[nuE_col]].first()

In [ ]:
print(nuE_per_slc)
print("len(nuE_per_slc) = %d" %len(nuE_per_slc))

In [ ]:
nuE_per_pfp

In [ ]:
#### 4.2) Draw a plot of nu.E for each slc
plt.hist(nuE_per_slc.E, bins=np.linspace(0., 6., 71), histtype="step", label=["all"])
plt.xlabel("Neutrino Energy (GeV)")
plt.ylabel(f"Neutrinos (POT = {target_POT:.2e})")
plt.legend()
plt.show()

In [ ]:
#### 4.3) Draw a plot of nu.E for each slc
###### We can see that some event are double counted where there are multiple pfp objects in a slc
plt.hist(nuE_per_pfp.E, bins=np.linspace(0., 6., 71), histtype="step", label=["all"])
plt.xlabel("Neutrino Energy (GeV)")
plt.ylabel(f"Neutrinos (POT = {target_POT:.2e})")
plt.legend()
plt.show()

In [ ]:
#### 4.4) Draw a plot of nu.E for each slc with POT scaling
plt.hist(nuE_per_slc.E, bins=np.linspace(0., 6., 71), weights=[np.ones_like(data) * POT_scale for data in nuE_per_slc.E], histtype="step", label=["all"])
plt.xlabel("Neutrino Energy (GeV)")
plt.ylabel(f"Neutrinos (POT = {target_POT:.2e})")
plt.legend()
plt.show()